In [1]:
import numpy as np
import pandas as pd
import datetime
from sklearn.utils import shuffle
%matplotlib inline

In [2]:
df = pd.read_csv("Data/openpowerlifting.csv")

C:\Users\tvaino\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
#Remove unnecessary columns
da = df[['Name', 'Sex', 'Event', 'Equipment', 'Age', 'Wilks', 'Tested', 'Date']]



In [4]:
# Changing column tested nan rows to no
da['Tested'] = da['Tested'].replace(np.nan,0)
da['Tested'] = da['Tested'].replace('Yes',1)
da['Sex'] = da['Sex'].replace('M',0)
da['Sex'] = da['Sex'].replace('F',1)

<ipython-input-4-479747ad4687>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  da['Tested'] = da['Tested'].replace(np.nan,0)
<ipython-input-4-479747ad4687>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  da['Tested'] = da['Tested'].replace('Yes',1)
<ipython-input-4-479747ad4687>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [5]:
# Rows where event is not SBD will be dropped
da = da[da.Event=='SBD']

In [6]:
#Drop rows where age is nan
da = da.dropna(how='any', subset=['Name','Sex', 'Equipment', 'Age', 'Wilks', 'Date'])

In [7]:
da['Equipment'] = da['Equipment'].replace('Raw',0)
da['Equipment'] = da['Equipment'].replace('Single-ply',1)
da = da[(da['Equipment']==1) | (da['Equipment']==0)]

In [8]:
#Adding column with approximate date of birth
da['BirthYear'] = da.apply(lambda row: int(str(row.Date)[:4]) - (row.Age), axis = 1)

In [9]:
da

,Name,Sex,Event,Equipment,Age,Wilks,Tested,Date,BirthYear
11,Christina Tsatsoulis,1,SBD,0,27.0,333.24,0,2018-10-27,1991.0
14,Elizabeth Ciortuz,1,SBD,0,19.0,290.95,0,2018-10-27,1999.0
19,Jasmine Hoare,1,SBD,0,31.0,353.56,0,2018-10-27,1987.0
21,Joelle Savgy,1,SBD,0,29.0,294.98,0,2018-10-27,1989.0
31,Nicole Brown,1,SBD,0,28.0,284.47,0,2018-10-27,1990.0
...,...,...,...,...,...,...,...,...,...
1423347,Adam Jasiński,0,SBD,0,65.5,372.98,1,2017-04-01,1951.5
1423348,Włodzimierz Żmijewski,0,SBD,0,67.5,367.91,1,2017-04-01,1949.5
1423349,Marian Cafalik,0,SBD,0,60.5,327.07,1,2017-04-01,1956.5
1423350,Marian Piwowarczyk,0,SBD,0,55.5,304.16,1,2017-04-01,1961.5


In [10]:
#Sort by date and names
da = da.sort_values(['Name','Date'])

In [11]:
da['PersonID'] = 0
da['PreviousComps'] = 0
p_id = 0
last_name = ""
last_birth_year = 0
last_sex = 2
competition_rate = 0

for i in range(len(da)):
    row = da.iloc[i]
    new_name = row['Name']
    new_birth_year = row['BirthYear']
    new_sex = row['Sex']
    if new_name == last_name:
        if new_sex == last_sex and abs(new_birth_year-last_birth_year)<=1:
            #print(new_name)
            competition_rate += 1
            da['PersonID'].iloc[i] = p_id
            da['PreviousComps'].iloc[i] = competition_rate
        else:
            competition_rate = 1
            last_name = new_name
            last_birth_year = new_birth_year
            last_sex = new_sex
            p_id+=1
            da['PersonID'].iloc[i] = p_id
            da['PreviousComps'].iloc[i] = competition_rate
    else:
        competition_rate = 1
        last_name = new_name
        last_birth_year = new_birth_year
        last_sex = new_sex
        p_id+=1
        da['PersonID'].iloc[i] = p_id
        da['PreviousComps'].iloc[i] = competition_rate
da.head(50)
        
    

C:\Users\tvaino\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,Name,Sex,Event,Equipment,Age,Wilks,Tested,Date,BirthYear,PersonID,PreviousComps
913534,A Abduzhabarov,0,SBD,1,23.5,312.90,1,2012-03-26,1988.5,1,1
1369333,A Ajeesha,1,SBD,0,16.5,295.29,1,2017-12-04,2000.5,2,1
1371420,A Ashwin,0,SBD,1,16.5,326.82,1,2012-05-01,1995.5,3,1
943185,A Bauer,0,SBD,1,20.5,380.09,1,1999-03-25,1978.5,4,1
1370862,A Dilmuradov,0,SBD,1,65.5,222.09,1,2004-04-28,1938.5,5,1
1373689,A Dilymuradov,0,SBD,1,69.5,203.39,1,2008-05-04,1938.5,6,1
943315,A Dzhaman,1,SBD,1,16.5,329.56,1,1999-03-25,1982.5,7,1
774343,A DziewiczKiewicz,1,SBD,1,21.5,424.06,1,2001-06-14,1979.5,8,1
944759,A Eginov,0,SBD,1,24.5,444.06,1,2000-09-27,1975.5,9,1
1029329,A Ernandos-Ortega,0,SBD,1,17.5,476.52,1,2002-09-25,1984.5,10,1


In [12]:
da = da.reset_index()

In [13]:
# Remove 
da = da[['PreviousComps', 'Sex', 'Equipment', 'Age', 'Wilks', 'Tested', 'Date']]

In [14]:
# Splitting data into test and train data set
da = shuffle(da, random_state=2)
split_id = int(len(da)*0.7)
da_train =da.iloc[:,:split_id]
da_test = da.iloc[split_id:]

In [15]:
# Saving clean data to file
da_test.to_csv(r'data/test_data.csv')
da_train.to_csv(r'data/train_data.csv')